<a href="https://colab.research.google.com/github/yasvanthika122/ML_Models/blob/main/Ada_booster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
#load data
test = pd.read_csv("https://raw.githubusercontent.com/ezioauditore-tech/AI/main/datasets/Titanic/test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/ezioauditore-tech/AI/main/datasets/Titanic/train.csv")

In [3]:
train.info(),test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

(None, None)

In [4]:
train.set_index("PassengerId",inplace=True)
test.set_index("PassengerId",inplace=True)
train.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S


In [5]:
y_train = train["Survived"]

In [6]:
train_test = train.append(test)
train_test.shape

<ipython-input-6-f36bb9a34d7a>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_test = train.append(test)


(1309, 11)

In [7]:
columns_to_drop = ["Name","Age","SibSp","Ticket","Cabin","Parch","Embarked"]#our wish
train_test.drop(labels=columns_to_drop,axis=1,inplace=True)
train_test

,Survived,Pclass,Sex,Fare
PassengerId,,,,
1,0.0,3,male,7.2500
2,1.0,1,female,71.2833
3,1.0,3,female,7.9250
4,1.0,1,female,53.1000
5,0.0,3,male,8.0500
...,...,...,...,...
1305,NaN,3,male,8.0500
1306,NaN,1,female,108.9000
1307,NaN,3,male,7.2500


In [8]:
train_test.isnull().sum()

Survived    418
Pclass        0
Sex           0
Fare          1
dtype: int64

In [9]:
train_test_dummies = pd.get_dummies(train_test,columns=["Sex"])
train_test_dummies

,Survived,Pclass,Fare,Sex_female,Sex_male
PassengerId,,,,,
1,0.0,3,7.2500,0,1
2,1.0,1,71.2833,1,0
3,1.0,3,7.9250,1,0
4,1.0,1,53.1000,1,0
5,0.0,3,8.0500,0,1
...,...,...,...,...,...
1305,NaN,3,8.0500,0,1
1306,NaN,1,108.9000,1,0
1307,NaN,3,7.2500,0,1


In [10]:
train_test_dummies.shape

(1309, 5)

In [11]:
train_test_dummies.fillna(value=1.0,inplace=True)
train_test_dummies.isnull().sum()

Survived      0
Pclass        0
Fare          0
Sex_female    0
Sex_male      0
dtype: int64

In [12]:
X_train=train_test_dummies.values[0:891]
X_test=train_test_dummies.values[891:]

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

In [14]:
from sklearn.model_selection import train_test_split

X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X_train_scale, y_train, random_state=0)

In [15]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [16]:
base_classifier = DecisionTreeClassifier(max_depth=1)
adaboost_classifier = AdaBoostClassifier(base_classifier, n_estimators=20, random_state=32)
adaboost_classifier.fit(X_train_sub, y_train_sub)
y_pred = adaboost_classifier.predict(X_test_sub)
accuracy = accuracy_score(y_test_sub, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0
